# Exploration 9. 추천 시스템  Recommender Sysyem

어느덧 추천시스템은 우리 생활 깊숙이 들어와 있다. 유튜브, 넷플릭스 등 동영상 플랫폼이 기가막히게 내가 좋아할 만한 콘텐츠를 찾아서 자동으로 플레이해 주고 있다.  

아마존, 쿠팡 온라인 쇼핑 사이트에서도 내 취향에 맞는 상품추천을 쉽게 접할 수 있다. 페이스북, 인스타그램 같은 SNS는 말할 것도 없고, 어쩌면 느끼지도 못하겠지만 뉴스나 광고까지도 추천시스템의 원리가 숨어들어 있다.

__나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해 주는 추천 시스템__ 을 구현해보자.

![image](https://user-images.githubusercontent.com/84179578/129284947-1bc13337-57c8-44be-a111-25d46d1009ea.png)

유저가 좋아하는 특정 아티스트와 유사한 다른 아티스트를 추천하는 추천시스템을 만들어 보려고 한다.   

Last.fm 에서 제공하는 어떤 유저가 특정 아티스트의 노래를 몇 번이나 들었는지에 대한 [데이터](http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html)를 사용한다.

## 0. 데이터 준비

이번에 사용할 데이터 형식인 `tsv`는 Tab-Separated Values의 약자로서, Comma-Separated Values의 약자인 csv 파일과 거의 동일하나 구분자만 comma가 아니라 tab('\t') 문자를 활용한 것이다.  

이런 텍스트 파일은 Pandas의 read_csv 메소드로도 충분히 파싱해서 열어볼 수 있다.

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# 사용하는 컬럼만 남김
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿈
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


## 1. 데이터 탐색

추천 모델을 만들기 전에 데이터의 다음과 같은 기본적인 정보를 확인해야한다.

- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

In [4]:
# 유저 수
data['user_id'].nunique()

358868

In [5]:
# 아티스트 수
data['artist'].nunique()

291346

In [6]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [7]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [8]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

## 2. 모델 검증을 위한 사용자 초기 정보 세팅

유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많다. 이와 동일한 과정이 필요하다.

만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야한다.  
일단은 아래와 같이 임의로 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠다.

In [9]:
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가 

data.tail(10)       # 잘 추가되었는지 확인

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


## 3. 모델에 활용하기 위한 전처리
데이터 관리를 쉽게 하기 위해 __indexing__ 작업을 하겠다.

`pandas.DataFrame.unique()` 은 특정 컬럼에 포함된 유니크한 데이터만 모아 주어 indexing 작업을 할때 매우 유용하다.

In [10]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [11]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 함
print(artist_to_idx['black eyed peas'])

358868
376


In [12]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


`data`의 `user_id`와 `artist` 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경된 것을 확인 할 수 있다.

### 사용자의 명시적/암묵적 평가  

명시적/암묵적 피드백 데이터셋에 대해 블로그에 정리하였다.

> 블로그 링크 추가 예정

In [13]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')

147740,17535660
Ratio of only_one over all data is 0.84%


앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용한다.
- 한 번이라도 들었으면 선호한다고 판단
- 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단

### CSR(Compressed Sparse Row) Matrix

In [14]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## 4. MF 모델 학습

이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.

- `implicit` 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지이다

- 이 패키지에 구현된 `als(AlternatingLeastSquares)` 모델을 사용한다.  
  Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있다.

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

AlternatingLeastSquares 클래스의 __init__ 파라미터
- 1. `factors` : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
- 2. `regularization` : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
- 3. `use_gpu` : GPU를 사용할 것인지 
- 4. `iterations` : epochs와 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 완료되었다. 이제 아래의 2가지 사항을 살펴보자.
- Zimin 벡터와 black eyed peas의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [20]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]
zimin_vector

array([-0.3816664 ,  0.1805846 ,  0.8234457 ,  1.0115271 ,  0.7510378 ,
       -0.46071187,  0.4782541 , -0.6079583 , -1.6379521 , -0.1629727 ,
       -0.55547017,  1.0443075 , -0.20650105, -0.3215918 ,  0.4691117 ,
       -0.2952599 , -0.6884578 , -0.04716763,  0.19848503,  0.5927813 ,
        0.35202563,  1.2214894 , -0.67429143, -0.3632305 ,  1.1729612 ,
        1.6098125 , -0.11955595,  0.8499455 , -0.48849764,  1.0128258 ,
       -0.75122905, -1.7227659 ,  1.0868298 , -0.67221415,  0.41645324,
       -0.14794609,  0.4267897 ,  0.66086483, -0.9002616 , -0.4692956 ,
       -0.39195678, -0.20275602,  0.36167863, -0.7511863 , -0.07608757,
        0.4260343 ,  0.9058962 , -0.3573681 ,  0.02639347,  0.61580354,
        0.7148055 , -0.2125434 , -0.1907274 ,  0.00471608, -0.2373114 ,
        1.0767345 , -0.7640293 ,  0.18967888,  0.45848745,  0.76327026,
        0.0647615 , -0.6503162 ,  0.63662887, -0.6668955 ,  0.75325674,
       -0.48096815,  0.86049336,  0.20816977,  0.82791024, -0.18

In [21]:
black_eyed_peas_vector

array([-0.00109966,  0.00879107,  0.01832983,  0.02917103,  0.00363794,
        0.00055176,  0.00407902, -0.00457211, -0.00893074,  0.00054724,
       -0.00052099,  0.00087051, -0.00720332,  0.00573421, -0.00726381,
        0.00295269, -0.00582861,  0.0179385 ,  0.01353916,  0.02075337,
        0.02093971,  0.01457254, -0.00466019, -0.01375905,  0.02280028,
        0.0368034 , -0.00607827,  0.01094029,  0.00726657,  0.01575765,
       -0.00418617, -0.01459836,  0.00132904,  0.00263809,  0.00266725,
        0.0085691 ,  0.01025606,  0.01719542, -0.00994561,  0.00869044,
        0.00574725, -0.00069512, -0.00572343,  0.00242739,  0.00719209,
        0.00398105,  0.03008783,  0.00040384, -0.01039599,  0.01341661,
        0.02359814,  0.00332574,  0.01092823, -0.0013619 ,  0.0023021 ,
        0.03430745, -0.00594443,  0.00300396,  0.00427881,  0.01675303,
        0.01277404,  0.01408083,  0.02143686,  0.00852029,  0.01044258,
        0.01184878,  0.00723678,  0.00903632,  0.0040429 , -0.00

In [22]:
# zimin과 black_eyed_peas를 내적
np.dot(zimin_vector, black_eyed_peas_vector)

0.5035893

1이 나와야 될 것 같은데 한참 낮은 수치인 0.5정도가 나왔다.   

factors를 늘리거나 iterations를 늘려야 할 것 같다.

그렇다면 모델이 zimin 의 `queen` 에 대한 선호도를 어떻게 예측하는지 보자.

In [23]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.30351448

## 5. 비슷한 아티스트 찾기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 아티스트를 찾아보자.  

처음으로 `coldplay` 로 찾아보겠다.

In [24]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 1.0),
 (277, 0.9868154),
 (28, 0.98390365),
 (5, 0.978611),
 (217, 0.9728681),
 (473, 0.9725395),
 (490, 0.97002393),
 (418, 0.9689724),
 (247, 0.96515274),
 (910, 0.95247),
 (694, 0.95140076),
 (55, 0.9502721),
 (1018, 0.94864124),
 (268, 0.945326),
 (782, 0.9402255)]

(아티스트의 id, 유사도) Tuple 로 반환하고 있다. 아티스트의 id를 다시 아티스트의 이름으로 매핑 시키자.

In [25]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'placebo',
 'oasis',
 'u2',
 'the beatles',
 'nirvana',
 'foo fighters',
 'arctic monkeys',
 'the smashing pumpkins',
 'pink floyd',
 'the white stripes']

여러번 반복하기위해 위의 과정을 함수로 만들자.

In [26]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

In [27]:
get_similar_artist('2pac')

['2pac',
 'dr. dre',
 'notorious b.i.g.',
 'snoop dogg',
 'the game',
 'nas',
 'jay-z',
 '50 cent',
 'ice cube',
 'ludacris']

힙합의 경우 마니아들이 많아서 더 잘 추천되는 것으로 보인다.

In [28]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'the pussycat dolls',
 'beyoncé',
 'christina aguilera',
 'justin timberlake',
 'kelly clarkson',
 'leona lewis']

lady gaga 의 경우 여자 아티스트들이 추천되는 것을 볼 수 있다.

## 6. 유저에게 아티스트 추천 

`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통하여 좋아할 만한 아티스트를 추천받을 수 있다.  

`filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument이다.

In [29]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.4805964),
 (369, 0.45835048),
 (550, 0.4581458),
 (1800, 0.43891737),
 (627, 0.434291),
 (354, 0.42255157),
 (2249, 0.42162645),
 (274, 0.41453478),
 (901, 0.41132665),
 (391, 0.40043685),
 (355, 0.39841235),
 (2902, 0.38547748),
 (5555, 0.38346362),
 (944, 0.38079917),
 (621, 0.37990648),
 (409, 0.37391263),
 (618, 0.37181944),
 (564, 0.35497972),
 (637, 0.35353118),
 (1777, 0.35195696)]

In [30]:
# 인덱스를 아티스트 이름으로 mapping
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'justin timberlake',
 'britney spears',
 'lady gaga',
 'maroon 5',
 'nelly furtado',
 'katy perry',
 'michael jackson',
 'pink',
 'christina aguilera',
 'madonna',
 'james blunt',
 'timbaland',
 'avril lavigne',
 'alicia keys',
 'amy winehouse',
 'the pussycat dolls',
 'kanye west',
 'akon',
 'mariah carey']

`AlternatingLeastSquares` 클래스에 구현된 `explain` 메소드를 사용하면 제가 기록을 남긴 데이터 중 __이 추천에 기여한 정도__를 확인할 수 있다.

In [31]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [32]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.21952166287378197),
 ('black eyed peas', 0.15318196124460345),
 ('jason mraz', 0.057801089362251735),
 ('coldplay', 0.05125664334232031),
 ('maroon5', -6.245423209550346e-05)]

추천시스템에서 Baseline으로 많이 사용되는 MF를 통해 유저에게 아티스트를 추천해보았다. 하지만 이번에 만든 모델은 몇 가지 아쉬운 점이 있다.
- 1. 유저, 아티스트에 대한 Meta 정보를 반영하기 쉽지 않다. 연령대별로 음악 취향이 굉장히 다를 가능성이 있다.
- 2. 유저가 언제 play 했는지에 반영하기 쉽지 않다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해야한다.